# Training of Model

## Import libraries

In [38]:
import pandas as pd

## Load data

In [39]:
trainD = pd.read_csv('Data/train.csv', parse_dates=['date'])
testD = pd.read_csv('Data/test.csv', parse_dates=['date'])
isHoliday = pd.read_csv('Data/holidays_events.csv', parse_dates=['date'])

In [40]:
trainD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [41]:
isHoliday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


## Filters Earthquake dates

In [42]:
trainDWOEarthQuake = trainD[(trainD['date'] < '2016-04-16') | (trainD['date'] >= '2016-07-16')]

## Adds Holiday column

In [43]:
#Drops the rows related to holidays that were transferred
isHoliday = isHoliday.drop(isHoliday[isHoliday.transferred == True].index)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         338 non-null    datetime64[ns]
 1   type         338 non-null    object        
 2   locale       338 non-null    object        
 3   locale_name  338 non-null    object        
 4   description  338 non-null    object        
 5   transferred  338 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 16.2+ KB


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [44]:
#New column which assigns true if the date is holiday or false if is not.
trainDWOEarthQuake = trainDWOEarthQuake.assign(is_holiday=False)

#Makes a list with the ordered dates in holidays
isHoliday = isHoliday.sort_values('date',ascending=True)
holiday_dates = isHoliday['date'].to_list()

len(holiday_dates)
i = 0

for index, row in trainDWOEarthQuake.iterrows():
    if row['date'].strftime('%Y-%m-%d') > holiday_dates[i].strftime('%Y-%m-%d'):
        i+=1
    if row['date'].strftime('%Y-%m-%d') == holiday_dates[i].strftime('%Y-%m-%d') and i < len(holiday_dates):
        trainDWOEarthQuake['is_holiday'].loc[index]=True

#trainDWOEarthQuake['is_holiday'] = trainDWOEarthQuake['date'].map(lambda x: x.date in isHoliday['date'].to_list())

C:\Users\an2fe\anaconda3v2\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Encoding

In [48]:
#Categorical encoding for variables with categories

#family
Xaux = trainDWOEarthQuake['family'].astype('category').cat.codes
trainDWOEarthQuake['family'] = Xaux

In [ ]:
#New column month of sale
trainDWOEarthQuake['month']=trainDWOEarthQuake.apply(lambda row: int(row['date'].strftime('%m')),axis=1)
